<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/ANOVA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [2]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
#df_train_A = df_train[df_train['Histology'] == 'adenocarcinoma']
#df_test_A = df_test[df_test['Histology'] == 'adenocarcinoma']

#df_train_L = df_train[df_train['Histology'] == 'large cell']
#df_test_L = df_test[df_test['Histology'] == 'large cell']

#df_train_S = df_train[df_train['Histology'] == 'squamous cell carcinoma']
#df_test_S = df_test[df_test['Histology'] == 'squamous cell carcinoma']

In [0]:
#public_data_A = df_train_A.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#public_data_L = df_train_L.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#public_data_S = df_train_S.drop(['Histology','Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
#PA_data_A = df_test_A.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#PA_data_L = df_test_L.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
#PA_data_S = df_test_S.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_data = df_train.drop(['Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)
PA_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_data_1 = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
public_labels = df_train.Histology

In [0]:
PA_labels = df_test.Histology

In [0]:
feat_A = public_data[public_data.Histology == 'adenocarcinoma']

In [0]:
feat_L = public_data[public_data.Histology == 'large cell']

In [0]:
feat_S = public_data[public_data.Histology == 'squamous cell carcinoma']

In [0]:
feat_A_1 = feat_A.drop(['Histology'], axis=1)
feat_L_1 = feat_L.drop(['Histology'], axis=1)
feat_S_1 = feat_S.drop(['Histology'], axis=1)


In [36]:
feat_A_1

,VoxelVolume,Maximum3DDiameter,MeshVolume,MajorAxisLength,Sphericity,LeastAxisLength,Elongation,SurfaceVolumeRatio,Maximum2DDiameterSlice,Flatness,SurfaceArea,MinorAxisLength,Maximum2DDiameterColumn,Maximum2DDiameterRow,GrayLevelVariance,HighGrayLevelEmphasis,DependenceEntropy,DependenceNonUniformity,GrayLevelNonUniformity,SmallDependenceEmphasis,SmallDependenceHighGrayLevelEmphasis,DependenceNonUniformityNormalized,LargeDependenceEmphasis,LargeDependenceLowGrayLevelEmphasis,DependenceVariance,LargeDependenceHighGrayLevelEmphasis,SmallDependenceLowGrayLevelEmphasis,LowGrayLevelEmphasis,JointAverage,SumAverage,JointEntropy,ClusterShade,MaximumProbability,Idmn,JointEnergy,Contrast,DifferenceEntropy,InverseVariance,DifferenceVariance,Idn,...,10Percentile,Kurtosis,Mean,ShortRunLowGrayLevelEmphasis,GrayLevelVariance.1,LowGrayLevelRunEmphasis,GrayLevelNonUniformityNormalized,RunVariance,GrayLevelNonUniformity.1,LongRunEmphasis,ShortRunHighGrayLevelEmphasis,RunLengthNonUniformity,ShortRunEmphasis,LongRunHighGrayLevelEmphasis,RunPercentage,LongRunLowGrayLevelEmphasis,RunEntropy,HighGrayLevelRunEmphasis,RunLengthNonUniformityNormalized,GrayLevelVariance.2,ZoneVariance,GrayLevelNonUniformityNormalized.1,SizeZoneNonUniformityNormalized,SizeZoneNonUniformity,GrayLevelNonUniformity.2,LargeAreaEmphasis,SmallAreaHighGrayLevelEmphasis,ZonePercentage,LargeAreaLowGrayLevelEmphasis,LargeAreaHighGrayLevelEmphasis,HighGrayLevelZoneEmphasis,SmallAreaEmphasis,LowGrayLevelZoneEmphasis,ZoneEntropy,SmallAreaLowGrayLevelEmphasis,Coarseness,Complexity,Strength,Contrast.1,Busyness
0,51905.377962,66.288317,51847.748274,50.574214,0.649258,37.884620,0.821088,0.199752,63.135672,0.749090,10356.675894,41.525899,65.067279,55.325619,12.481889,1084.854684,5.949646,918.046673,5312.127441,0.122582,97.652454,0.050648,141.807349,0.124010,42.380287,163828.301666,0.000266,0.001208,33.190463,66.380925,4.331076,-475.448161,0.308299,0.994676,0.133643,8.029403,2.169232,0.424312,6.065116,0.967009,...,-75.0,17.777521,5.650502,0.001166,17.478141,0.001372,0.192540,1.399893,2288.112500,3.749302,793.234742,6675.859117,0.776723,4220.221337,0.654950,0.003806,4.209293,1049.544424,0.560736,34.869500,42116.076135,0.060025,0.517739,1145.238698,132.775769,42183.224231,540.316964,0.122035,36.510775,4.877236e+07,751.520796,0.747563,0.002453,5.741322,0.001663,0.000357,928.016789,1.153806,0.020920,1.306338
1,13432.502747,58.057539,13312.697411,48.354803,0.572236,18.135097,0.546338,0.356577,40.146103,0.375042,4747.006589,26.418066,32.760898,56.652510,60.615944,1076.589137,7.130906,595.667519,351.846858,0.307871,239.202712,0.126873,32.011715,0.031426,12.612334,41890.348882,0.001812,0.003214,33.522040,67.044080,7.487967,-2829.110940,0.055759,0.985695,0.015596,38.337756,3.408960,0.322015,23.246287,0.935189,...,-397.0,5.224099,-91.310969,0.003404,65.432452,0.003524,0.064158,0.230922,262.139314,1.567042,926.829706,3237.676584,0.908183,1777.017297,0.869059,0.004164,5.072184,1044.275778,0.790377,84.615342,598.216508,0.033349,0.543736,843.878866,51.757732,607.367912,543.633876,0.330564,0.465530,8.137725e+05,763.567010,0.764879,0.006307,6.451087,0.004959,0.001680,2944.805484,2.266070,0.146173,0.253533
2,25843.872675,52.918217,25724.437234,39.406474,0.675497,28.487740,0.891907,0.242519,46.415213,0.722920,6238.658603,35.146929,47.180420,46.322906,55.064124,1131.900166,6.932158,844.783490,1023.136953,0.253452,192.059998,0.093605,57.855402,0.045979,22.286238,79266.271357,0.000812,0.001864,34.046184,68.092368,6.796695,-3133.512010,0.106695,0.984515,0.033042,32.375097,3.182144,0.362633,21.028555,0.936889,...,-363.0,5.387644,-67.724986,0.001973,61.733697,0.002096,0.085594,0.426431,626.125860,1.968083,906.575851,5266.417163,0.870954,2393.005796,0.809077,0.002930,4.968901,1079.567623,0.718481,68.928494,3157.415098,0.036760,0.531612,1319.991542,91.276279,3170.626259,525.709823,0.275125,2.306913,4.377395e+06,732.877970,0.757392,0.003456,6.294554,0.002520,0.000745,2146.780339,1.238883,0.152919,0.611772
3,22152.7

## Perform one-way ANOVA with scipy

ANOVA Hypotheses

    1) Null hypotheses: Groups means are equal (no variation in means of groups)
    2) Alternative hypotheses: At least, one group mean is different from other groups


ANOVA Assumptions

    1) Residuals (experimental error) are normally distributed (Shapiro Wilks Test)
    2) Homogeneity of variances (variances are equal between treatment groups) (Levene or Bartlett Test)
    3) Observations are sampled independently from each other


In [15]:
# load packages
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns F and P-value
fvalue, pvalue = stats.f_oneway(feat_A, feat_L, feat_S)
print(fvalue, pvalue)

4.7184035638853965 0.010540043241126989


In [16]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue, pvalue = stats.f_oneway(feat_A, feat_L, feat_S)

  if pvalue <= 0.05:
    print(fvalue, pvalue, column)


3.260231291649289 0.0415894832636264 HighGrayLevelEmphasis
4.7184035638853965 0.010540043241126989 Maximum
5.359338541197532 0.005818378887600915 Range
3.635718433558213 0.029123760736867767 LongRunHighGrayLevelEmphasis
3.187154836462757 0.04458597869558036 HighGrayLevelRunEmphasis
4.652836901674387 0.011204051595582876 GrayLevelVariance.2
3.1553089945578137 0.04595955606831947 SmallAreaHighGrayLevelEmphasis
4.845308726734106 0.009366122195029402 ZoneEntropy


Null hypotheses: Groups means are equal (no variation in means of groups)


Alternative hypotheses: At least, one group mean is different from other groups

Soglia che si considera in genere: 0.05

se pvalue <= 0.005 si rigetta l'Hp zero


##Compute the Kruskal-Wallis H-test for independent samples.

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA

In [0]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue, pvalue = stats.kruskal(feat_A, feat_L, feat_S)

  if pvalue <= 0.05:
    print(fvalue, pvalue, column)


6.147661147655405 0.04624367533780151 Maximum2DDiameterRow
7.327260213194336 0.02563927030450879 HighGrayLevelEmphasis
6.621721927698957 0.03648474817830056 Idn
6.620504649169675 0.036506960987710696 Autocorrelation
6.444153541861307 0.0398721668601435 Median
6.809322290739744 0.033218074273387084 Maximum
7.715868105731206 0.021111569887821632 90Percentile
8.575330349975955 0.013736961284395596 Range
6.975503981330917 0.03056951554313273 LongRunHighGrayLevelEmphasis
6.768094930660595 0.03390992734838747 HighGrayLevelRunEmphasis
11.13192988155447 0.0038258870091253303 GrayLevelVariance.2
7.892243014763665 0.019329526002898275 SmallAreaHighGrayLevelEmphasis
7.1864424655768175 0.0275095728076037 HighGrayLevelZoneEmphasis
10.247500028441095 0.005953654690921986 ZoneEntropy


In [0]:
for column in public_data_1.columns:

  feat_A = public_data[column][public_data.Histology == 'adenocarcinoma']
  feat_L = public_data[column][public_data.Histology == 'large cell']
  feat_S = public_data[column][public_data.Histology == 'squamous cell carcinoma']

  fvalue_1, pvalue_1 = stats.f_oneway(feat_A, feat_L, feat_S)
  fvalue_2, pvalue_2 = stats.kruskal(feat_A, feat_L, feat_S)


  if (pvalue_1 <= 0.05) & (pvalue_2 <= 0.5):
    print(column)

HighGrayLevelEmphasis
Maximum
Range
LongRunHighGrayLevelEmphasis
HighGrayLevelRunEmphasis
GrayLevelVariance.2
SmallAreaHighGrayLevelEmphasis
ZoneEntropy


From ANOVA analysis, we know that treatment differences are statistically significant, but ANOVA does not tell which treatments are significantly different from each other. To know the pairs of significant different treatments, we will perform multiple pairwise comparison (Post-hoc comparison) analysis using Tukey HSD test.

In [0]:
# load packages
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [0]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=public_data['Maximum'], groups=public_data['Histology'], alpha=0.05)
print(m_comp)

              Multiple Comparison of Means - Tukey HSD, FWER=0.05               
    group1              group2         meandiff p-adj    lower    upper   reject
--------------------------------------------------------------------------------
adenocarcinoma              large cell  16.1107    0.9 -267.4243 299.6456  False
adenocarcinoma squamous cell carcinoma 244.6389 0.1118  -42.6893 531.9671  False
    large cell squamous cell carcinoma 228.5282 0.0131   39.9145 417.1419   True
--------------------------------------------------------------------------------


Above results from Tukey HSD suggests that L-S comparison rejects null hypothesis and indicates statistical significant differences.

##Test ANOVA assumptions

The Shapiro-Wilk test can be used to check the normal distribution of residuals.


Null hypothesis: data is drawn from normal distribution.

In [0]:
not_normal_features_A = []
not_normal_features_L = []
not_normal_features_S = []


for column in public_data_1.columns:

  w_A, pvalue_A = stats.shapiro(feat_A_1[column])
  w_L, pvalue_L = stats.shapiro(feat_L_1[column])
  w_S, pvalue_S = stats.shapiro(feat_S_1[column])


  if pvalue_A <= 0.05:
    #print(pvalue, column)
    not_normal_features_A.append(column)
  
  if pvalue_L <= 0.05:
    #print(pvalue, column)
    not_normal_features_L.append(column)

  if pvalue_S <= 0.05:
    #print(pvalue, column)
    not_normal_features_S.append(column)




In [41]:
print(len(not_normal_features_A), len(not_normal_features_L), len(not_normal_features_S))

49 87 75


In [28]:
len(public_data_1.columns)

107

89/107 features non sono distribuite normalmente